<a href="https://colab.research.google.com/github/ppotatoo/Capstone-Predictive-Modeling-for-Increasing-Adoption-Probability/blob/master/logistic_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
import statsmodels.api as sm
import pathlib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/캡스톤2_제주유기견/설문결과-Raw-Data.csv',sep=',')

In [5]:
df.columns = ['인식매체','경험','인식','관심도','지식','구성원동의','환경','비용지불의사','정부지원정책','犬성별','중성화','나이','털길이','품종','색상','크기','성격','공고기간','입양희망여부','d1','d2','조사 나이대','성별','d5','d6']
df

,인식매체,경험,인식,관심도,지식,구성원동의,환경,비용지불의사,정부지원정책,犬성별,...,크기,성격,공고기간,입양희망여부,d1,d2,조사 나이대,성별,d5,d6
0,1,1,2,3,4,2,2,5,4,2,...,3,2,4,1,4,1,61,1,3,NaN
1,1,1,4,5,4,4,5,4,4,1,...,4,4,4,2,1,2,26,1,3,NaN
2,1,1,2,4,4,2,2,4,4,1,...,3,2,3,2,3,1,46,1,3,NaN
3,1,2,3,2,2,3,2,2,2,2,...,3,4,4,3,3,1,53,2,6,NaN
4,1,2,3,2,2,3,3,4,4,2,...,3,3,3,2,2,1,41,2,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1,2,3,2,3,3,2,2,4,2,...,3,3,3,2,3,1,30,2,3,NaN
296,1,1,5,5,3,4,4,4,4,2,...,3,3,3,3,3,2,67,2,4,NaN
297,1,1,3,4,4,4,3,4,4,1,...,2,3,3,3,1,3,45,1,3,NaN
298,1,1,2,3,1,2,3,2,1,2,...,1,1,1,3,2,2,43,1,3,NaN


In [6]:
real_df = df.drop(['d1','d2','d5','d6'], axis = 1) # 라이프 스타일, 직업 삭제
real_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   인식매체    300 non-null    int64
 1   경험      300 non-null    int64
 2   인식      300 non-null    int64
 3   관심도     300 non-null    int64
 4   지식      300 non-null    int64
 5   구성원동의   300 non-null    int64
 6   환경      300 non-null    int64
 7   비용지불의사  300 non-null    int64
 8   정부지원정책  300 non-null    int64
 9   犬성별     300 non-null    int64
 10  중성화     300 non-null    int64
 11  나이      300 non-null    int64
 12  털길이     300 non-null    int64
 13  품종      300 non-null    int64
 14  색상      300 non-null    int64
 15  크기      300 non-null    int64
 16  성격      300 non-null    int64
 17  공고기간    300 non-null    int64
 18  입양희망여부  300 non-null    int64
 19  조사 나이대  300 non-null    int64
 20  성별      300 non-null    int64
dtypes: int64(21)
memory usage: 49.3 KB


In [7]:
idx = real_df[real_df["입양희망여부"] == 3].index     #  인덱스의 값이 저장됨
real_df.drop(idx , inplace=True) # 해당 인덱스를 제거함

In [19]:
old_range = {'경험':{2:0}, '인식매체':{2:0}, '입양희망여부' :{2:0}, '성별' : {2:0}, '犬성별' : {2:0}} 
# 0 입양안한다, 1 한다,    성별 - 0 = 여자, 1 = 남자 , 인식매체 0 오프라인, 1 온라인,  경험 0 아니오, 1 예
logi_df = real_df.replace(old_range)
logi_df

,인식매체,경험,인식,관심도,지식,구성원동의,환경,비용지불의사,정부지원정책,犬성별,...,나이,털길이,품종,색상,크기,성격,공고기간,입양희망여부,조사 나이대,성별
0,1,1,2,3,4,2,2,5,4,0,...,4,4,4,4,3,2,4,1,61,1
1,1,1,4,5,4,4,5,4,4,1,...,4,3,4,3,4,4,4,0,26,1
2,1,1,2,4,4,2,2,4,4,1,...,3,3,4,3,3,2,3,0,46,1
4,1,0,3,2,2,3,3,4,4,0,...,3,3,4,4,3,3,3,0,41,0
5,1,1,4,5,4,5,3,5,4,1,...,3,2,4,4,2,3,5,1,29,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,1,1,3,4,4,4,4,5,5,1,...,4,3,4,3,4,3,4,1,52,1
291,1,0,2,3,3,2,2,2,4,1,...,3,3,3,3,3,2,3,0,26,0
292,1,1,2,3,3,4,4,3,3,1,...,4,4,5,5,4,3,4,1,33,1
295,1,0,3,2,3,3,2,2,4,0,...,3,3,4,4,3,3,3,0,30,0


# 로지스틱 회귀분석 (종속변수 = 입양희망여부)

In [9]:
logi_df.describe().T #입양 희망여부 평균 0.5 => 결과변수 균형잡힘

,count,mean,std,min,25%,50%,75%,max
인식매체,204.0,0.901961,0.298099,0.0,1.00,1.0,1.0,1.0
경험,204.0,0.720588,0.449814,0.0,0.00,1.0,1.0,1.0
인식,204.0,3.299020,0.964288,1.0,3.00,3.0,4.0,5.0
관심도,204.0,3.411765,1.126072,1.0,3.00,4.0,4.0,5.0
지식,204.0,3.117647,0.995498,1.0,2.00,3.0,4.0,5.0
구성원동의,204.0,3.181373,1.162402,1.0,2.00,3.0,4.0,5.0
환경,204.0,2.774510,1.152482,1.0,2.00,3.0,4.0,5.0
비용지불의사,204.0,3.147059,1.198497,1.0,2.00,3.0,4.0,5.0
정부지원정책,204.0,3.651961,1.069711,1.0,3.00,4.0,4.0,5.0
犬성별,204.0,0.475490,0.500627,0.0,0.00,0.0,1.0,1.0


In [10]:
logi_df.max()

인식매체       1
경험         1
인식         5
관심도        5
지식         5
구성원동의      5
환경         5
비용지불의사     5
정부지원정책     5
犬성별        1
중성화        5
나이         5
털길이        5
품종         5
색상         5
크기         5
성격         5
공고기간       5
입양희망여부     1
조사 나이대    68
성별         1
dtype: int64

In [11]:
logi_df["입양희망여부"].value_counts() # 0 입양x, 1 입양 o

1    102
0    102
Name: 입양희망여부, dtype: int64

In [12]:
x = logi_df.drop('입양희망여부', axis=1)  #변수 지정
y = logi_df['입양희망여부']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [13]:
print(len(x_train))
print(len(x_test))

142
62


In [14]:
log = LogisticRegression() #로지스틱 회귀분석 시행

log.fit(x_train, y_train) #모델의 정확도 확인
print('학습용 데이터셋 정확도 : %.2f' % log.score(x_train, y_train))
print('검증용 데이터셋 정확도 : %.2f' % log.score(x_test, y_test))

학습용 데이터셋 정확도 : 0.90
검증용 데이터셋 정확도 : 0.84


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
from sklearn.metrics import classification_report
y_pred=log.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86        35
           1       0.81      0.81      0.81        27

    accuracy                           0.84        62
   macro avg       0.84      0.84      0.84        62
weighted avg       0.84      0.84      0.84        62



In [16]:
logit = sm.Logit(logi_df[['입양희망여부']],x) #로지스틱 회귀분석 시행
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.398215
         Iterations 7


In [17]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 입양희망여부   No. Observations:                  204
Model:                          Logit   Df Residuals:                      184
Method:                           MLE   Df Model:                           19
Date:                Tue, 06 Dec 2022   Pseudo R-squ.:                  0.4255
Time:                        05:23:14   Log-Likelihood:                -81.236
converged:                       True   LL-Null:                       -141.40
Covariance Type:            nonrobust   LLR p-value:                 9.614e-17
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
인식매체          -0.9384      0.687     -1.366      0.172      -2.285       0.408
경험             0.5971      0.575      1.039      0.299      -0.529       1.723
인식            -0.5529      0.288     -1.922      0.055      -1.117       0.011
관심도           -0.0885      0.301     -0.294      0.769      -0.679       0.502
지식             0.4533      0.299      1.516      0.130      -0.133       1.039
구성원동의          0.5132      0.316      1.623      0.105      -0.107       1.133
환경             0.5801      0.267      2.174      0.030       0.057       1.103
비용지불의사         1.3106      0.328      3.997      0.000       0.668       1.953
정부지원정책        -0.1451      0.273     -0.531      0.596      -0.681       0.391
犬성별            0.3516      0.455      0.773      0.440      -0.540       1.243
중성화           -0.2704      0.245     -1.104      0.270      -0.750       0.210
나이            -0.4158      0.350     -1.188      0.235      -1.102       0.270
털길이            0.2606      0.334      0.781      0.435      -0.393       0.914
품종            -0.4636      0.350     -1.324      0.185      -1.150       0.223
색상             0.1608      0.340      0.473      0.636      -0.505       0.827
크기            -0.1551      0.291     -0.533      0.594      -0.725       0.415
성격            -0.2704      0.327     -0.828      0.408      -0.911       0.370
공고기간           0.0251      0.364      0.069      0.945      -0.689       0.739
조사 나이대        -0.0523      0.019     -2.803      0.005      -0.089      -0.016
성별             0.2209      0.488      0.453      0.651      -0.735       1.177
==============================================================================
"""

In [18]:
np.exp(result.params) #오즈비가 1 기준으로 큰지 작은지 파악 1과 값이 수렴한다면 영향 x

인식매체      0.391267
경험        1.816803
인식        0.575275
관심도       0.915296
지식        1.573461
구성원동의     1.670608
환경        1.786233
비용지불의사    3.708559
정부지원정책    0.864919
犬성별       1.421401
중성화       0.763058
나이        0.659795
털길이       1.297665
품종        0.628984
색상        1.174440
크기        0.856357
성격        0.763066
공고기간      1.025380
조사 나이대    0.949022
성별        1.247144
dtype: float64